In [5]:
import pandas as pd
import copy

# Node의 부모 Class
class Node:
    def __init__(self, node_name):
        self.node_name = node_name

# Edge의 부모 Class
class Edge:
    def __init__(self, origin, dest):
        self.origin = origin
        self.dest = dest

# Airport Class
class Airport(Node):
    def __init__(self, airport_name):
        super().__init__(airport_name)

# Flight Class
class Flight(Edge):
    def __init__(self, index, origin, dest, origin_date, origin_time, dest_date, dest_time, flight_duration, flight_type):
        super().__init__(origin, dest)
        self.index = index
        self.origin_date = origin_date
        self.origin_time = origin_time
        self.dest_date = dest_date
        self.dest_time = dest_time
        self.flight_duration = flight_duration
        self.flight_type = flight_type

    def print(self):
        print(self.index, self.origin, self.origin_date, self.origin_time, self.dest,
              self.dest_date, self.dest_time, self.flight_duration, self.flight_type)

# Stack Node의 구조
class Stackitem:
    def __init__(self, flight: Flight, flights_sequence: list = None):
        if flights_sequence:
            self.flight = flight
            flights_sequence.append(flight.index)
            self.flights_sequence = flights_sequence
            # self.time = time + dt.timedelta(flight.flight_duration)
        else:
            self.flight = flight
            self.flights_sequence = [flight.index]
            # self.time = flight.flight_duration;

   Flight Data Unnamed: 1           Unnamed: 2   Unnamed: 3 Unnamed: 4  \
0        INDEX     ORIGIN          ORIGIN_DATE  ORIGIN_TIME       DEST   
1           F1        LAX  2023-02-01 00:00:00     09:00:00        ORD   
2           F2        LAX  2023-02-02 00:00:00     09:00:00        ORD   
3           F3        LAX  2023-02-03 00:00:00     09:00:00        ORD   
4           F4        LAX  2023-02-04 00:00:00     13:00:00        HNL   
5           F5        LAX  2023-02-02 00:00:00     13:00:00        HNL   
6           F6        LAX  2023-02-03 00:00:00     13:00:00        HNL   
7           F7        ORD  2023-02-01 00:00:00     17:00:00        LAX   
8           F8        ORD  2023-02-05 00:00:00     17:00:00        LAX   
9           F9        ORD  2023-02-05 00:00:00     17:00:00        LAX   
10         F10        ORD  2023-02-02 00:00:00     15:00:00        HNL   
11         F11        ORD  2023-02-03 00:00:00     15:00:00        HNL   
12         F12        ORD  2023-02-04 

In [ ]:
# Initial Setting - Test
flight_graph = dict()
flight_excel = pd.read_excel("CGinputEX.xlsx")
flight_excel.columns = ['INDEX', 'ORIGIN', 'ORIGIN_DATE','ORIGIN_TIME', 'DEST','DEST_DATE', 'DEST_TIME', 'FLIGHT_DURATION','AIRCRAFT_TYPE' ]
airports = ["HNL","ORD","LAX"]

# Graph insert Node
for airport_name in airports :
    flight_graph[airport_name] = list()


# Graph insert Edge
for i in range(flight_excel.shape[0]) :
    flight_graph[flight_excel.loc[i]['ORIGIN']].append(Flight(flight_excel.loc[i]['INDEX'], flight_excel.loc[i]['ORIGIN'], flight_excel.loc[i]['DEST'],
                                                              flight_excel.loc[i]['ORIGIN_DATE'], flight_excel.loc[i]['ORIGIN_TIME'], flight_excel.loc[i]['DEST_DATE'], flight_excel.loc[i]['DEST_TIME'],
                                                              flight_excel.loc[i]['FLIGHT_DURATION'], flight_excel.loc[i]['AIRCRAFT_TYPE']))

# print(flight_excel)
# for airport_name in airports : 
#     print(airport_name, "- start - ---------Edge----------")
#     for flight in flight_graph[airport_name] :
#         flight.print()